In [ ]:
!pip install transformers datasets

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install --upgrade transformers[torch]


In [ ]:
!pip install accelerate>=0.21.0

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade transformers[torch]  accelerate


In [ ]:
!pip install wandb loralib


In [ ]:
import wandb
import datasets
import numpy as np
# from huggingface_peft import PEFTForCausalLM, LoRAConfig
from peft import LoraConfig, get_peft_model

In [ ]:
dataset = load_dataset("ZahrizhalAli/mental_health_conversational_dataset")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset=pd.DataFrame(dataset)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   175 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


In [ ]:
from sklearn.model_selection import train_test_split
train_test = train_test_split(dataset, test_size=0.2)  # 80/20 split
train_dataset = train_test[0]
eval_dataset = train_test[1]

In [ ]:
train_dataset.info()
eval_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 101 to 39
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   140 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 153 to 111
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   35 non-null     object
dtypes: object(1)
memory usage: 560.0+ bytes


In [ ]:
train_dataset.dtypes

train    object
dtype: object

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

In [ ]:
# Configuration
model_name = "gpt2"
output_dir = './results'
lora_rank = 4
lora_alpha = 8


In [ ]:
train_dataset[0]

{'train': {'text': '<HUMAN>: How Does Hypnotherapy Work?\n<ASSISTANT>: Hypnotherapy is usually considered an aid to psychotherapy (counselling), rather than a treatment in itself. It helps with psychotherapy because the hypnotic state allows people to enter a deep state of relaxation that can better allow them to explore painful thoughts, feelings, and memories they might have hidden from their conscious minds. In addition, hypnosis enables people to perceive some things differently, such as blocking an awareness of pain.\n\nHypnotherapy can be used in two ways, as suggestion therapy or for patient analysis.\n\n1. Suggestion therapy: The hypnotic state makes the person better able to respond to suggestions. Therefore, hypnotherapy can help some people stop certain behaviours, such as smoking or nail-biting.\n\n2. Analysis: This approach was historically used to induce a relaxed and uninhibited state to find suspected unconscious causes of a disorder or symptom, such as a traumatic past

In [ ]:
# 3. Load Model and Tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token



In [ ]:
train_dataset.__getitem__(0)

{'train': {'text': '<HUMAN>: How Does Hypnotherapy Work?\n<ASSISTANT>: Hypnotherapy is usually considered an aid to psychotherapy (counselling), rather than a treatment in itself. It helps with psychotherapy because the hypnotic state allows people to enter a deep state of relaxation that can better allow them to explore painful thoughts, feelings, and memories they might have hidden from their conscious minds. In addition, hypnosis enables people to perceive some things differently, such as blocking an awareness of pain.\n\nHypnotherapy can be used in two ways, as suggestion therapy or for patient analysis.\n\n1. Suggestion therapy: The hypnotic state makes the person better able to respond to suggestions. Therefore, hypnotherapy can help some people stop certain behaviours, such as smoking or nail-biting.\n\n2. Analysis: This approach was historically used to induce a relaxed and uninhibited state to find suspected unconscious causes of a disorder or symptom, such as a traumatic past

In [ ]:
# 4. Preprocessing

# train_dict = train_dataset.to_dict()

# def preprocess_function(examples):
#     inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
#     return inputs

# train_tokenized_dataset = preprocess_function(train_dict)


train_list = train_dataset.to_list()

def preprocess_function(examples):
    inputs = tokenizer(examples['train'], padding='max_length', truncation=True, max_length=512)
    return inputs

train_tokenized_dataset = preprocess_function(train_list[0])

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
train_dataset

Dataset({
    features: ['train', '__index_level_0__'],
    num_rows: 140
})

In [ ]:
def preprocess_function(samples):
    # tokenize inputs
    samples = [i['text'] for i in samples['train']]
    model_inputs = tokenizer(samples, truncation=True, max_length = 128)
    return model_inputs

tokenized_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['train', '__index_level_0__'])


Map:   0%|          | 0/140 [00:00<?, ? examples/s]

In [ ]:
eval_tokenized= eval_dataset.map(preprocess_function, batched=True, remove_columns=['train', '__index_level_0__'])


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 140
})

In [ ]:
# 5. Model with PEFT for LoRA
lora_config = LoraConfig(r=lora_rank, lora_alpha=lora_alpha)
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Computing Perplexity

In [ ]:



def compute_perplexity(eval_pred):
  logits, labels = eval_pred
  shift_logits = logits[..., :-1, :].contiguous()
  shift_labels = labels[..., 1:].contiguous()
  loss_fct = torch.nn.CrossEntropyLoss(ignore_index=1000) # Assuming -100 is your padding index
  loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
  perplexity = torch.exp(loss)
  return {"perplexity": perplexity}



In [ ]:
# 6. Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=50,
    learning_rate=2e-5,
    warmup_steps=100,
    weight_decay=0.01,
    fp16=True,
    report_to = ["wandb"],
    save_total_limit=2,  # Saving a few checkpoints
)

In [ ]:
# 7. WandB Setup
wandb.init(project="lora-text-generation")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Trainer
import os
# os.environ["WANDB_DISABLED"] = "false"
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_tokenized,
    data_collator=data_collator,
    compute_metrics=compute_perplexity  # Adding perplexity computation function
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=105, training_loss=3.213523410615467, metrics={'train_runtime': 8.4247, 'train_samples_per_second': 49.853, 'train_steps_per_second': 12.463, 'total_flos': 27483226767360.0, 'train_loss': 3.213523410615467, 'epoch': 3.0})

In [ ]:
trainer.save_model(output_dir)
